# Excel ↔ PDF/Image Comparator (Notebook)
Upload two files in this runtime, extract table from PDF/Image, compare to Excel/CSV, and export CSV reports.

In [ ]:

import pandas as pd
import pdfplumber
from PIL import Image
try:
    import pytesseract
    TESS_AVAILABLE = True
except Exception:
    TESS_AVAILABLE = False

def normalize_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]) or df[c].dtype == "object":
            df[c] = df[c].astype(str).str.strip()
    return df

def read_structured(path: str) -> pd.DataFrame:
    if path.lower().endswith(".csv"):
        df = pd.read_csv(path, dtype=str, keep_default_na=False)
    else:
        df = pd.read_excel(path, dtype=str)
    return normalize_df(df)

def extract_tables_from_pdf(pdf_path: str):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            try:
                candidates = page.extract_tables(table_settings={
                    "vertical_strategy": "lines",
                    "horizontal_strategy": "lines",
                    "intersection_tolerance": 5
                })
            except Exception:
                candidates = page.extract_tables()
            for tbl in candidates or []:
                if not tbl or len(tbl) < 2:
                    continue
                header = [str(h).strip() if h is not None else f"col_{i}" for i, h in enumerate(tbl[0])]
                rows = [[("" if x is None else str(x).strip()) for x in r] for r in tbl[1:]]
                width = max(len(header), *(len(r) for r in rows)) if rows else len(header)
                header = (header + [f"col_{i}" for i in range(len(header), width)])[:width]
                rows = [ (r + [""] * (width - len(r)))[:width] for r in rows ]
                df = pd.DataFrame(rows, columns=header)
                df = normalize_df(df)
                tables.append(df)
    return tables

def extract_table_from_image(img_path: str):
    if not TESS_AVAILABLE:
        return []
    img = Image.open(img_path)
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DATAFRAME)
    if "line_num" not in data.columns or data.empty:
        return []
    data = data.dropna(subset=["text"])
    data = data[data["text"].str.strip() != ""]
    if data.empty:
        return []
    group_cols = ["page_num", "block_num", "par_num", "line_num"]
    lines = (
        data.sort_values(["page_num", "block_num", "par_num", "line_num", "word_num"])
            .groupby(group_cols)["text"]
            .apply(lambda toks: " ".join([str(t) for t in toks]))
            .reset_index()
    )
    if lines.empty:
        return []
    df = pd.DataFrame({"ocr_line": lines["text"].astype(str).str.strip()})
    df = df[df["ocr_line"] != ""]
    if df.empty:
        return []
    return [df.reset_index(drop=True)]

def combine_tables(tables):
    tables = [t for t in tables if t is not None and not t.empty]
    if not tables:
        return None
    cleaned = []
    for df in tables:
        df = df.copy()
        df.columns = [str(c).strip() for c in df.columns]
        keep_cols = [c for c in df.columns if not (df[c].astype(str).str.strip() == "").all()]
        if keep_cols:
            df = df[keep_cols]
        cleaned.append(df)
    picked = max(cleaned, key=lambda d: d.shape[0] * max(1, d.shape[1]))
    return normalize_df(picked)

def pick_key_columns(dfA: pd.DataFrame, dfB: pd.DataFrame):
    common = [c for c in dfA.columns if c in dfB.columns]
    if not common:
        return []
    preferred = [c for c in common if c.lower() in {"id", "key", "sku"} or c.lower().endswith("_id")]
    if preferred:
        return preferred
    candidate = []
    n = max(1, len(dfA))
    for c in common:
        try:
            uniq = dfA[c].nunique(dropna=True) / n
            if uniq > 0.8:
                candidate.append(c)
        except Exception:
            pass
    if candidate:
        return candidate[:3]
    return common[:3]

def compare_dataframes(dfA: pd.DataFrame, dfB: pd.DataFrame):
    dfA = normalize_df(dfA)
    dfB = normalize_df(dfB)
    keys = pick_key_columns(dfA, dfB)
    if not keys:
        common_cols = [c for c in dfA.columns if c in dfB.columns]
        left = dfA[common_cols].assign(_src="A")
        right = dfB[common_cols].assign(_src="B")
        merged = left.merge(right[common_cols], how="outer", indicator=True)
        missing_in_B = merged[merged["_merge"] == "left_only"][common_cols]
        missing_in_A = merged[merged["_merge"] == "right_only"][common_cols]
        value_mismatches = pd.DataFrame(columns=["column", "value_A", "value_B"])
        return missing_in_B, missing_in_A, value_mismatches, keys
    left_only = dfA.merge(dfB[keys].drop_duplicates(), on=keys, how="left", indicator=True)
    missing_in_B = left_only[left_only["_merge"] == "left_only"].drop(columns=["_merge"])
    right_only = dfB.merge(dfA[keys].drop_duplicates(), on=keys, how="left", indicator=True)
    missing_in_A = right_only[right_only["_merge"] == "left_only"].drop(columns=["_merge"])
    a_idx = dfA.set_index(keys); b_idx = dfB.set_index(keys)
    shared_index = a_idx.index.intersection(b_idx.index).drop_duplicates()
    a_aligned = a_idx.loc[shared_index]; b_aligned = b_idx.loc[shared_index]
    common_cols = [c for c in a_aligned.columns if c in b_aligned.columns and c not in keys]
    mismatch_records = []
    for idx in shared_index:
        idx_tuple = (idx,) if not isinstance(idx, tuple) else idx
        rowA = a_aligned.loc[idx]; rowB = b_aligned.loc[idx]
        for col in common_cols:
            va = "" if col not in rowA else ("" if pd.isna(rowA[col]) else str(rowA[col]))
            vb = "" if col not in rowB else ("" if pd.isna(rowB[col]) else str(rowB[col]))
            if va != vb:
                rec = {"column": col, "value_A": va, "value_B": vb}
                for i, k in enumerate(keys):
                    rec[k] = idx_tuple[i]
                mismatch_records.append(rec)
    value_mismatches = pd.DataFrame(mismatch_records)
    if not value_mismatches.empty:
        value_mismatches = value_mismatches[[*keys, "column", "value_A", "value_B"]]
    return missing_in_B, missing_in_A, value_mismatches, keys
print("Helpers ready.")    


## Usage
1. Put your Excel/CSV at a path like `data.xlsx`
2. Put your PDF or image at a path like `table.pdf` or `table.png`
3. Run the cell below with your filenames.

In [ ]:

# Example:
excel_path = "data.xlsx"   # change me
other_path = "table.pdf"   # change me

df_struct = read_structured(excel_path)
if other_path.lower().endswith(".pdf"):
    tables = extract_tables_from_pdf(other_path)
else:
    tables = extract_table_from_image(other_path)
extracted = combine_tables(tables)

missing_in_B, missing_in_A, value_mismatches, keys_used = compare_dataframes(df_struct, extracted)

print("Detected keys:", keys_used)
print("Missing in PDF/Image:", len(missing_in_B))
print("Missing in Excel:", len(missing_in_A))
print("Value mismatches:", len(value_mismatches))

# Save reports
if len(missing_in_B): missing_in_B.to_csv("missing_in_pdf_image.csv", index=False)
if len(missing_in_A): missing_in_A.to_csv("missing_in_excel.csv", index=False)
if len(value_mismatches): value_mismatches.to_csv("value_mismatches.csv", index=False)
print("Done.")
